In [1]:
using POMDPs
using Random # for AbstractRNG
using POMDPModelTools
using Pkg

using DataFrames
Pkg.add("JSON")

POMDPs.add_registry()
using Pkg; Pkg.add("DiscreteValueIteration")

using DiscreteValueIteration

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
   Cloning registry from "https://github.com/JuliaPOMDP/Registry"
Fetching: [========================================>]  100.0 %.0 %=========>                               ]  21.4 %]  42.7 %==========================>              ]  64.1 %]  85.3 % [=====================================>   ]  91.7 %

┌ Info: registry `JuliaPOMDP` already exists in `~/.julia/registries`
└ @ POMDPs /Users/efan/.julia/packages/POMDPs/JiYXY/src/utils.jl:65


 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [2]:
using Pkg
Pkg.add("StaticArrays")
using POMDPSimulators
using POMDPPolicies

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [ ]:

include("./ParallelChains.jl")

In [ ]:
num_chains = 5
num_states = 6
r_mean = 5.0
r_std = 2.0

true_values = DataFrame(theta = [r_mean for i in 1:num_chains],
                        std = [r_std  for i in 1:num_chains])

In [ ]:
using POMDPSimulators
using POMDPPolicies
rng = MersenneTwister(1235);

PParallelChainMDP() = PParallelChainMDP(num_states+1,num_chains, .9,

        true_values.theta,
        true_values.std,
        MersenneTwister(1235))
m = PParallelChainMDP()

# policy that maps every input to an action
policy = FunctionPolicy(s->3)
for i in 1:3
    for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
      render(m, (s,a,r))
      println("s,a,r:($s,$a,$r)")
    end
end

In [ ]:
curry(f, a) = (xs...) -> f(a, xs...)

In [ ]:
using POMDPSimulators
using POMDPPolicies
rng = MersenneTwister(1235);

num_chains = 10
num_states = 30
r_mean = 5.0
r_std = 2.0
@requirements_info ValueIterationSolver() PParallelChainMDP(num_states+1,num_chains, .9,

        randn(rng, Float32, num_chains) .* r_std .+ r_mean)

In [ ]:
num_chains=2
r_mean = 0
r_std = 1
## sigma is 1
## Reward is noisy ~ N(theta_c, sigma^2)
## Initial prior mean is N(theta_c, sigma^2 + c)
rng = MersenneTwister(1235)
th_c = [randn(rng, Float64, 1)[1] * (100 + i) for i in 1:num_chains]
println(th_c)
true_values = DataFrame(theta = th_c,
                         std = [r_std  for i in 1:num_chains])
mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        true_values.theta,
        true_values.std, 
        rng)
# @requirements_info ValueIterationSolver() mdp
solver = ValueIterationSolver(max_iterations=100, belres=1e-6, include_Q=true, verbose=true) # initializes the Solver type
vip = solve(solver, mdp) # runs value iterations


In [ ]:
vip.qmat

In [ ]:
LinearIndices((num_chains, num_states))[1,1]

In [ ]:

for i in 1:num_chains
    println("q value for (1,1),$i is $(vip.qmat[LinearIndices((num_chains, num_states))[1,1],i])")
    println("q value for $((i,num_states)),1 is $(vip.qmat[LinearIndices((num_chains, num_states))[i,num_states],1])")
end

In [ ]:
mdp.Rs

In [ ]:
using POMDPSimulators
using POMDPPolicies



for (s, a, r) in stepthrough(mdp, vip, "s,a,r", max_steps=40)
    render(mdp, (s,a,r))
    println("s,a,r:($s,$a,$r)")
end



In [3]:
function ucb_pol(li, policy, priors, i, actions, s)
    best_action = action(policy, s)
    #println("in state: $s, best action:$best_action")
    return action(policy, s)
end

ucb_pol (generic function with 1 method)

In [4]:
curry(f, x) = (xs...) -> f(x, xs...)
function ucb_mdp_builder(rng, true_mdp, priors, i, num_states, num_chains, steps, start_state)
  # build mdp
  new_means = priors.theta + priors.std
  mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        [priors.theta[j] + 4* priors.std[j] for j in 1:num_chains])
  solver = ValueIterationSolver(max_iterations=1000, belres=1e-6, include_Q=true)#, verbose=true) # initializes the Solver type
  vip = solve(solver, mdp)
  li = LinearIndices((num_chains, num_states))
  new_policy = FunctionPolicy(curry(curry(curry(curry(curry(ucb_pol, li), vip), priors),i), actions))
  if start_state == nothing
    return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, "s,a,r,sp,t",
                              max_steps=steps))
  else
     return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, start_state, "s,a,r,sp,t",
                              max_steps=steps))       
  end
end

ucb_mdp_builder (generic function with 1 method)

In [5]:

include("./ParallelChains.jl")
using Distributions
function do_runs(nruns, epochs, num_agents, num_chains, num_states, H, update_priors_check, update_priors,
                 mdp_iter_builder, is_thompson_sampling, base, rng)
    # UCB

    # setup constants here

    # So best idea:
    # for ucb:
    # we generate an mdp with mu + sigma rewards (priors) for each end node.
    #  we collect history until the criteria in PAC-EXPLORE then update the prior
    # 
    # for thompson sampling
    # at each time step (might skip this for this problem and save for max rew path) 
    #   take rewards and use to create a posterior.
    # for seed sampling
    # at beginning of episode
    #   each agent generates a new random "seed" 
    #   at each time step
    #     each agent generates a new mdp based on a deterministic mapping from seed to rewards (which also takes
    #              history into account)
    # this needs to update the actual reward.

    ## PC notes
    ## theta_c ~ N(0, 100 + c)
    r_mean = 0
    r_std = 1
    ## sigma is 1
    ## Reward is noisy ~ N(theta_c, sigma^2)
    ## Initial prior mean is N(theta_c, sigma^2 + c)
    runs = []
    true_values_list = DataFrame( run = [],
                                 c = [],
                                 theta = [],
                                 std = [])
    for run in 1:nruns
        th_c = [randn(rng, Float64, 1)[1] * sqrt(base + i) for i in 1:num_chains]
        #th_c[1] = 300
        println(th_c)
        th_c_sd = sqrt(var(th_c))
        r_std = th_c_sd
        true_values = DataFrame( run = [run for i in 1:num_chains],
                                 c = [i for i in 1:num_chains],
                                 theta = th_c,
                                 std = [r_std  for i in 1:num_chains])
        print("true mdp: $true_values")
        #priors = DataFrame(theta = [th_c[i] for i in 1:num_chains],
        priors = DataFrame(theta = [0 for i in 1:num_chains],
                           std = [(sqrt(r_std^2) + i) for i in 1:num_chains],
                           state = [(chain,num_states) for chain in 1:num_chains])
        print("priors: $priors")
        mdp = PParallelChainMDP(num_states+1,num_chains, .9,
                true_values.theta)
        do_update_priors() =  false
        # priors = randn(rng, Float32, num_chains) .* (r_std + c) .+ r_mean
        hist = run_chain!(
                   mdp_iter_builder=mdp_iter_builder,
                   true_mdp=mdp,
                   do_update_priors=update_priors_check,
                   update_priors=update_priors,
                   priors=priors,
                   true_vals=true_values,
                   n_agents=num_agents,
                   num_states=num_states,
                   num_chains=num_chains,
                   epochs=epochs,
                   steps=H,
                   is_thompson_sampling=is_thompson_sampling,
                   rng=rng) #,
                   #rev_action_map=rev_action_map)
        #print(Q_tables)
        println("(e,i,t,st,r)")
        hist
        for (e, ag, t, st, r)  in hist
          push!(runs, (run, e, ag, t, st, r)) 
        end
        append!(true_values_list, true_values)

    end
    return (true_values_list, runs)
end

do_runs (generic function with 1 method)

In [ ]:
num_agents = 100
num_chains = 10
num_states = 10
epochs = 1
nruns = 1
H = 30
do_update_priors() = true
true_values_list, runs = do_runs(nruns, 1, num_agents, num_chains, num_states, H, do_update_priors,
                                 ucb_update_priors, ucb_mdp_builder, false, 100)
true_values_list, runs

chain_len = num_states
num_chains = num_chains
ucb_regret = get_average_regret(runs, chain_len, num_chains, true_values_list)

In [6]:

using DataFrames
using Statistics
function ucb_update_priors(priors, hist, true_vals)
    # each state must be visted me times to update the prior
    # me is # of S*D^2 s= states, D =diameter (https://www.aaai.org/ocs/index.php/AAAI/AAAI15/paper/view/9542/9919)
    #     (D defined in http://www.jmlr.org/papers/volume11/jaksch10a/jaksch10a.pdf)
    ## ^^^ Actually, think should just do this statistically, and if n > 2, then we can adjust the prior.
    df = DataFrame(epoch = [x[1] for x in hist], agent=[x[2] for x in hist],
               time = [x[3] for x in hist], chain = [x[4][1] for x in hist], chain_state = [x[4][2] for x in hist],
        reward = [x[5] for x in hist])
    new_theta = []
    new_std = []
    new_state = []
    for arow in eachrow(priors)
        if nrow(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:]) > 1
            mu_p = arow.theta
            true_std = true_vals[arow.Row,:].std[1]
            s_p_2 = arow.std^2
            mu_s = mean(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:].reward)
            s_s_2 = var(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:].reward)
            cnt = nrow(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:])
            #tmp_theta =  (s_p_2/(s_s_2 + s_p_2)) * mu_s + (s_s_2/(s_s_2 + s_p_2)) * mu_p
            #tmp_std = 1/(1/s_p_2 + 1/s_s_2)
            tmp_theta =  (s_p_2/(true_std/cnt + s_p_2)) * mu_s + (s_s_2/(true_std/cnt + s_p_2)) * mu_p
            tmp_std = 1/(1/s_p_2 + cnt/true_std)
        else
            tmp_theta =  arow.theta
            tmp_std = arow.std
        end
        push!(new_state, arow.state)
        push!(new_theta, tmp_theta)
        push!(new_std, tmp_std)
    end
    new_priors = DataFrame(theta=new_theta, std=new_std, state=new_state)
    return new_priors
end



ucb_update_priors (generic function with 1 method)

In [ ]:
println(priors)
ucb_update_priors(priors, hist)

In [7]:
function get_average_regret(results, chain_len, num_chains, theta)
    df = DataFrame(run = [x[1] for x in results], epoch = [x[2] for x in results], agent=[x[3] for x in results],
               time = [x[4] for x in results], c = [x[5][1] for x in results], state = [x[5][2] for x in results],
               reward = [x[6] for x in results])
    df_R = df[df.reward.!==0,:]
    th_max = by(theta, :run, :theta => maximum)
    th_min = by(theta, :run, :theta => minimum)
    theta_joined = join(join(theta, th_max, on = :run), th_min, on = :run)
    df_joined = join(df_R, theta_joined, on = [:run,:c] )
    # needs to be max theta
    df_joined.Regret = df_joined.theta_maximum - df_joined.reward
    df_joined
end

get_average_regret (generic function with 1 method)

In [ ]:
true_values_list

In [ ]:
# Thompson sampling
num_agents = 100
num_chains = 10
num_states = 10
epochs = 1
nruns = 1
H = 30
do_update_priors() = true
true_values_list, runs = do_runs(nruns, 1, num_agents, num_chains, num_states, H, do_update_priors,
           ucb_update_priors, ucb_mdp_builder, true, 100)
true_values_list, runs

chain_len = num_states
num_chains = num_chains
thompson_regret = get_average_regret(runs, chain_len, num_chains, true_values_list)

In [8]:
curry(f, x) = (xs...) -> f(x, xs...)
function ssgn_mdp_builder(rng, true_mdp, priors, i, num_states, num_chains, steps, start_state)
  # build mdp
  # new_means = priors.theta + priors.std
  mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        #[randn(rng, Float32,1)[1] *  1 + priors.theta[j] for j in 1:num_chains])
        [randn(rng, Float32,1)[1] *  (2*priors.std[j]) + priors.theta[j] for j in 1:num_chains])
  solver = ValueIterationSolver(max_iterations=1000, belres=1e-6, include_Q=true)#, verbose=true) # initializes the Solver type
  vip = solve(solver, mdp)
  li = LinearIndices((num_chains, num_states))
  new_policy = FunctionPolicy(curry(curry(curry(curry(curry(ucb_pol, li), vip), priors),i), actions))
  if start_state == nothing
    return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, "s,a,r,sp,t",
                              max_steps=steps))
  else
     return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, start_state, "s,a,r,sp,t",
                              max_steps=steps))       
  end
end

ssgn_mdp_builder (generic function with 1 method)

In [ ]:
# Gaussian normal sampling
num_agents = 100
num_chains = 10
num_states = 10
epochs = 1
nruns = 1
H = 30
do_update_priors() = true
true_values_list, runs = do_runs(nruns, 1, num_agents, num_chains, num_states, H, do_update_priors,
    ucb_update_priors, ssgn_mdp_builder, false, 100)
true_values_list, runs

In [ ]:

chain_len = num_states
num_chains = num_chains
seed_regret = get_average_regret(runs, chain_len, num_chains, true_values_list)

In [ ]:
thompson_regret.name = ["Thompson Sampling" for i in 1:nrow(thompson_regret)]

In [ ]:
seed_regret.name = ["Seed Sampling" for i in 1:nrow(seed_regret)]

In [ ]:
ucb_regret.name = ["UCB" for i in 1:nrow(ucb_regret)]

In [ ]:
using CSV
all_regret = deepcopy(thompson_regret)
append!(all_regret,seed_regret)
append!(all_regret,ucb_regret)
CSV.write("pc_simulation.csv", all_regret)
all_regret

In [9]:

include("./ParallelChains.jl")
# Full loop
full_df = nothing
base = 100

rng = MersenneTwister()
for num_agents in [1,10,100,1000]
    println("*****************number of agents:$num_agents*******************")
    num_chains = 10
    num_states = 5
    epochs = 1
    nruns = 30
    H = 12
    do_update_priors() = true
    for run_type in ["Thompson Sampling", "Seed Sampling", "UCB"]
        println("------------------Run Type:$run_type------------------------")
        if run_type == "Thompson Sampling"
            true_values_list, runs = do_runs(nruns, epochs, num_agents, num_chains, num_states, H,
                                             do_update_priors, ucb_update_priors,  ssgn_mdp_builder, true,
                                             base, rng)
        elseif run_type == "UCB"
            true_values_list, runs = do_runs(nruns, epochs, num_agents, num_chains, num_states, H,
                                             do_update_priors, ucb_update_priors, ucb_mdp_builder, false,
                                             base, rng)
        else
            true_values_list, runs = do_runs(nruns, epochs, num_agents, num_chains, num_states, H,
                                             do_update_priors, ucb_update_priors,  ssgn_mdp_builder, false,
                                             base, rng)

        end

        chain_len = num_states
        num_chains = num_chains
        regret = get_average_regret(runs, chain_len, num_chains, true_values_list)
        regret.name = [run_type for i in 1:nrow(regret)]
        regret.num_agents = [num_agents for i in 1:nrow(regret)]
        regret.base = [base for i in 1:nrow(regret)]
        if full_df == nothing
            full_df = regret
        else
            append!(full_df,regret)
        end
    end
end 

println("***********************Done!!!*********************************")





*****************number of agents:1*******************
------------------Run Type:Thompson Sampling------------------------
[21.0953, -2.15157, 6.38098, 4.49676, -3.54297, -15.2009, -0.869341, -2.54323, -4.57697, 15.3809]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta     │ std     │
│     │ Int64 │ Int64 │ Float64   │ Float64 │
├─────┼───────┼───────┼───────────┼─────────┤
│ 1   │ 1     │ 1     │ 21.0953   │ 10.4607 │
│ 2   │ 1     │ 2     │ -2.15157  │ 10.4607 │
│ 3   │ 1     │ 3     │ 6.38098   │ 10.4607 │
│ 4   │ 1     │ 4     │ 4.49676   │ 10.4607 │
│ 5   │ 1     │ 5     │ -3.54297  │ 10.4607 │
│ 6   │ 1     │ 6     │ -15.2009  │ 10.4607 │
│ 7   │ 1     │ 7     │ -0.869341 │ 10.4607 │
│ 8   │ 1     │ 8     │ -2.54323  │ 10.4607 │
│ 9   │ 1     │ 9     │ -4.57697  │ 10.4607 │
│ 10  │ 1     │ 10    │ 15.3809   │ 10.4607 │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├─────┼───────┼─────────┼─────────┤
│ 1   │ 0     │ 11.4607 │

[15.7064, -10.0105, 6.77056, -16.8593, 7.61815, 6.69975, -12.8716, -0.617709, 10.6427, 7.35285]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta     │ std     │
│     │ Int64 │ Int64 │ Float64   │ Float64 │
├─────┼───────┼───────┼───────────┼─────────┤
│ 1   │ 7     │ 1     │ 15.7064   │ 11.0175 │
│ 2   │ 7     │ 2     │ -10.0105  │ 11.0175 │
│ 3   │ 7     │ 3     │ 6.77056   │ 11.0175 │
│ 4   │ 7     │ 4     │ -16.8593  │ 11.0175 │
│ 5   │ 7     │ 5     │ 7.61815   │ 11.0175 │
│ 6   │ 7     │ 6     │ 6.69975   │ 11.0175 │
│ 7   │ 7     │ 7     │ -12.8716  │ 11.0175 │
│ 8   │ 7     │ 8     │ -0.617709 │ 11.0175 │
│ 9   │ 7     │ 9     │ 10.6427   │ 11.0175 │
│ 10  │ 7     │ 10    │ 7.35285   │ 11.0175 │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├─────┼───────┼─────────┼─────────┤
│ 1   │ 0     │ 12.0175 │ (1, 5)  │
│ 2   │ 0     │ 13.0175 │ (2, 5)  │
│ 3   │ 0     │ 14.0175 │ (3, 5)  │
│ 4   │ 0     │ 15.0175 │ (4, 5)  │
│ 5   │

e: 1, t: 1, agent 1, result: (s = [7, 4], a = 1, r = 0, sp = [7, 5], t = 1)
e: 1, t: 1, agent 1, result: (s = [7, 5], a = 1, r = 3.6264697721199997, sp = [7, 6], t = 1)
(e,i,t,st,r)
[9.29766, -11.3693, -8.37233, -2.85568, -5.98362, 16.7208, -2.97405, -4.80143, 10.0936, 3.62006]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta    │ std     │
│     │ Int64 │ Int64 │ Float64  │ Float64 │
├─────┼───────┼───────┼──────────┼─────────┤
│ 1   │ 17    │ 1     │ 9.29766  │ 9.15965 │
│ 2   │ 17    │ 2     │ -11.3693 │ 9.15965 │
│ 3   │ 17    │ 3     │ -8.37233 │ 9.15965 │
│ 4   │ 17    │ 4     │ -2.85568 │ 9.15965 │
│ 5   │ 17    │ 5     │ -5.98362 │ 9.15965 │
│ 6   │ 17    │ 6     │ 16.7208  │ 9.15965 │
│ 7   │ 17    │ 7     │ -2.97405 │ 9.15965 │
│ 8   │ 17    │ 8     │ -4.80143 │ 9.15965 │
│ 9   │ 17    │ 9     │ 10.0936  │ 9.15965 │
│ 10  │ 17    │ 10    │ 3.62006  │ 9.15965 │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├─────┼───────┼──

(e,i,t,st,r)
[16.5459, -6.01454, -5.02086, 6.88223, -10.3877, -10.6524, -5.70438, 3.38743, 6.63543, 6.9244]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta    │ std     │
│     │ Int64 │ Int64 │ Float64  │ Float64 │
├─────┼───────┼───────┼──────────┼─────────┤
│ 1   │ 27    │ 1     │ 16.5459  │ 9.06281 │
│ 2   │ 27    │ 2     │ -6.01454 │ 9.06281 │
│ 3   │ 27    │ 3     │ -5.02086 │ 9.06281 │
│ 4   │ 27    │ 4     │ 6.88223  │ 9.06281 │
│ 5   │ 27    │ 5     │ -10.3877 │ 9.06281 │
│ 6   │ 27    │ 6     │ -10.6524 │ 9.06281 │
│ 7   │ 27    │ 7     │ -5.70438 │ 9.06281 │
│ 8   │ 27    │ 8     │ 3.38743  │ 9.06281 │
│ 9   │ 27    │ 9     │ 6.63543  │ 9.06281 │
│ 10  │ 27    │ 10    │ 6.9244   │ 9.06281 │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├─────┼───────┼─────────┼─────────┤
│ 1   │ 0     │ 10.0628 │ (1, 5)  │
│ 2   │ 0     │ 11.0628 │ (2, 5)  │
│ 3   │ 0     │ 12.0628 │ (3, 5)  │
│ 4   │ 0     │ 13.0628 │ (4, 5)  │
│ 5   │ 

[17.6986, 6.06646, -6.17169, 2.96506, 10.343, 5.72295, 2.21891, 9.20935, -27.091, 2.28226]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta    │ std     │
│     │ Int64 │ Int64 │ Float64  │ Float64 │
├─────┼───────┼───────┼──────────┼─────────┤
│ 1   │ 6     │ 1     │ 17.6986  │ 12.0715 │
│ 2   │ 6     │ 2     │ 6.06646  │ 12.0715 │
│ 3   │ 6     │ 3     │ -6.17169 │ 12.0715 │
│ 4   │ 6     │ 4     │ 2.96506  │ 12.0715 │
│ 5   │ 6     │ 5     │ 10.343   │ 12.0715 │
│ 6   │ 6     │ 6     │ 5.72295  │ 12.0715 │
│ 7   │ 6     │ 7     │ 2.21891  │ 12.0715 │
│ 8   │ 6     │ 8     │ 9.20935  │ 12.0715 │
│ 9   │ 6     │ 9     │ -27.091  │ 12.0715 │
│ 10  │ 6     │ 10    │ 2.28226  │ 12.0715 │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├─────┼───────┼─────────┼─────────┤
│ 1   │ 0     │ 13.0715 │ (1, 5)  │
│ 2   │ 0     │ 14.0715 │ (2, 5)  │
│ 3   │ 0     │ 15.0715 │ (3, 5)  │
│ 4   │ 0     │ 16.0715 │ (4, 5)  │
│ 5   │ 0     │ 17.0715 │

...

e: 1, t: 2, agent 1, result: (s = [10, 2], a = 1, r = 0, sp = [10, 3], t = 2)
e: 1, t: 3, agent 1, result: (s = [10, 3], a = 1, r = 0, sp = [10, 4], t = 3)
e: 1, t: 4, agent 1, result: (s = [10, 4], a = 1, r = 0, sp = [10, 5], t = 4)
e: 1, t: 5, agent 1, result: (s = [10, 5], a = 1, r = -5.287493829704651, sp = [10, 6], t = 5)
(e,i,t,st,r)
[2.47782, 9.58857, -1.86455, 10.9887, -5.05507, 11.9659, -2.123, -9.03741, 21.2201, 9.13461]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta    │ std     │
│     │ Int64 │ Int64 │ Float64  │ Float64 │
├─────┼───────┼───────┼──────────┼─────────┤
│ 1   │ 7     │ 1     │ 2.47782  │ 9.3528  │
│ 2   │ 7     │ 2     │ 9.58857  │ 9.3528  │
│ 3   │ 7     │ 3     │ -1.86455 │ 9.3528  │
│ 4   │ 7     │ 4     │ 10.9887  │ 9.3528  │
│ 5   │ 7     │ 5     │ -5.05507 │ 9.3528  │
│ 6   │ 7     │ 6     │ 11.9659  │ 9.3528  │
│ 7   │ 7     │ 7     │ -2.123   │ 9.3528  │
│ 8   │ 7     │ 8     │ -9.03741 │ 9.3528  │
│ 9   │ 7     │ 9     │ 21.2201  │ 9.3528  │


e: 1, t: 3, agent 1, result: (s = [10, 3], a = 1, r = 0, sp = [10, 4], t = 3)
e: 1, t: 4, agent 1, result: (s = [10, 4], a = 1, r = 0, sp = [10, 5], t = 4)
e: 1, t: 5, agent 1, result: (s = [10, 5], a = 1, r = 11.3540761039297, sp = [10, 6], t = 5)
(e,i,t,st,r)
[9.84824, -4.69614, 12.8426, -2.43393, 7.85782, 6.23499, 5.26108, 3.59586, 0.926083, -3.15059]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta    │ std     │
│     │ Int64 │ Int64 │ Float64  │ Float64 │
├─────┼───────┼───────┼──────────┼─────────┤
│ 1   │ 12    │ 1     │ 9.84824  │ 5.87124 │
│ 2   │ 12    │ 2     │ -4.69614 │ 5.87124 │
│ 3   │ 12    │ 3     │ 12.8426  │ 5.87124 │
│ 4   │ 12    │ 4     │ -2.43393 │ 5.87124 │
│ 5   │ 12    │ 5     │ 7.85782  │ 5.87124 │
│ 6   │ 12    │ 6     │ 6.23499  │ 5.87124 │
│ 7   │ 12    │ 7     │ 5.26108  │ 5.87124 │
│ 8   │ 12    │ 8     │ 3.59586  │ 5.87124 │
│ 9   │ 12    │ 9     │ 0.926083 │ 5.87124 │
│ 10  │ 12    │ 10    │ -3.15059 │ 5.87124 │priors: 10×3 DataFrame
│ Row │ the

e: 1, t: 5, agent 1, result: (s = [10, 5], a = 1, r = 1.4852433468351243, sp = [10, 6], t = 5)
(e,i,t,st,r)
[-11.0165, -4.88904, -0.536352, 1.59404, -5.81919, 0.591809, -12.2942, 8.93147, -6.42058, -17.9185]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta     │ std     │
│     │ Int64 │ Int64 │ Float64   │ Float64 │
├─────┼───────┼───────┼───────────┼─────────┤
│ 1   │ 17    │ 1     │ -11.0165  │ 7.79784 │
│ 2   │ 17    │ 2     │ -4.88904  │ 7.79784 │
│ 3   │ 17    │ 3     │ -0.536352 │ 7.79784 │
│ 4   │ 17    │ 4     │ 1.59404   │ 7.79784 │
│ 5   │ 17    │ 5     │ -5.81919  │ 7.79784 │
│ 6   │ 17    │ 6     │ 0.591809  │ 7.79784 │
│ 7   │ 17    │ 7     │ -12.2942  │ 7.79784 │
│ 8   │ 17    │ 8     │ 8.93147   │ 7.79784 │
│ 9   │ 17    │ 9     │ -6.42058  │ 7.79784 │
│ 10  │ 17    │ 10    │ -17.9185  │ 7.79784 │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├─────┼───────┼─────────┼─────────┤
│ 1   │ 0     │ 8.79784 │ (1, 5)  │
│ 2

e: 1, t: 5, agent 1, result: (s = [10, 5], a = 1, r = -16.83904430436806, sp = [10, 6], t = 5)
(e,i,t,st,r)
[5.97338, 10.5361, -2.24861, -0.00769868, 2.50102, -4.89584, -1.54046, -9.41132, 0.562307, 1.36859]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta       │ std     │
│     │ Int64 │ Int64 │ Float64     │ Float64 │
├─────┼───────┼───────┼─────────────┼─────────┤
│ 1   │ 27    │ 1     │ 5.97338     │ 5.52068 │
│ 2   │ 27    │ 2     │ 10.5361     │ 5.52068 │
│ 3   │ 27    │ 3     │ -2.24861    │ 5.52068 │
│ 4   │ 27    │ 4     │ -0.00769868 │ 5.52068 │
│ 5   │ 27    │ 5     │ 2.50102     │ 5.52068 │
│ 6   │ 27    │ 6     │ -4.89584    │ 5.52068 │
│ 7   │ 27    │ 7     │ -1.54046    │ 5.52068 │
│ 8   │ 27    │ 8     │ -9.41132    │ 5.52068 │
│ 9   │ 27    │ 9     │ 0.562307    │ 5.52068 │
│ 10  │ 27    │ 10    │ 1.36859     │ 5.52068 │priors: 10×3 DataFrame
│ Row │ theta │ std     │ state   │
│     │ Int64 │ Float64 │ Tuple…  │
├─────┼───────┼─────────┼─────────┤
│ 1   │ 0    

ArgumentError: ArgumentError: column name :Row not found in the data frame

In [10]:
using CSV
full_df[full_df[:Regret] .< 0.0,:Regret] = 0.0
CSV.write("pc_simulation.csv", full_df)
full_df

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1186


,run,epoch,agent,time,c,state,reward,theta,std,theta_maximum,theta_minimum,Regret,name,num_agents,base
,Int64,Int64,Int64,Int64,Int64,Int64,Real,Any,Any,Float64,Float64,Float64,String,Int64,Int64
1,1,1,1,1,10,5,15.5314,15.3809,10.4607,21.0953,-15.2009,5.56391,Thompson Sampling,1,100
2,2,1,1,1,8,5,2.76355,3.43095,7.99888,14.1967,-15.1683,11.4332,Thompson Sampling,1,100
3,3,1,1,1,9,5,5.25906,4.58597,11.3893,13.3382,-26.9935,8.07915,Thompson Sampling,1,100
4,4,1,1,1,7,5,13.1459,14.2802,12.1949,14.2802,-20.7871,1.1343,Thompson Sampling,1,100
5,5,1,1,1,9,5,5.87947,6.89705,8.45268,23.8504,-3.20658,17.9709,Thompson Sampling,1,100
6,6,1,1,1,9,5,-5.64067,-6.14409,7.7732,3.18182,-21.5737,8.8225,Thompson Sampling,1,100
7,7,1,1,1,10,5,6.57571,7.35285,11.0175,15.7064,-16.8593,9.13073,Thompson Sampling,1,100
8,8,1,1,1,10,5,-3.07938,-3.27474,12.1301,18.7678,-23.9254,21.8471,Thompson Sampling,1,100
9,9,1,1,1,8,5,16.3754,15.5381,7.16661,15.5381,-10.7157,0.0,Thompson Sampling,1,100


In [ ]:

full_df[full_df.name.=="Seed Sampling",:]

In [ ]:

full_df[(full_df.name.=="UCB").&(full_df.run.==1),:]

In [ ]:

show(full_df[(full_df.name.=="UCB").&(full_df.num_agents.==10),:], allrows=true, allcols=true)



In [ ]:
mean(full_df[full_df.name.=="UCB",:].Regret)